In [18]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-05-26 19:41:26--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6814 (6.7K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   6.65K  --.-KB/s    in 0s      

2022-05-26 19:41:26 (98.5 MB/s) - ‘utils.py’ saved [6814/6814]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [20]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer

In [6]:
BATCH_SIZE = 128
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16

In [7]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [8]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [ ]:
for color, weight in zip(df['color'], class_weight.values()):
  print(color, "=>", weight)

beige => 17.94
black => 13.8
blue => 15.47
brown => 17.92
cream => 56.81
crimson => 76.43
gold => 56.81
green => 21.2
grey => 12.41
navy-blue => 58.77
orange => 22.37
red => 16.39
silver => 20.17
titanium => 56.81
white => 2.97
yellow => 20.68


In [ ]:
class_weight

{0: 17.94,
 1: 13.8,
 2: 15.47,
 3: 17.92,
 4: 56.81,
 5: 76.43,
 6: 56.81,
 7: 21.2,
 8: 12.41,
 9: 58.77,
 10: 22.37,
 11: 16.39,
 12: 20.17,
 13: 56.81,
 14: 2.97,
 15: 20.68}

In [ ]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [ ]:
bit_model_url = 'https://tfhub.dev/google/bit/m-r50x1/1'
bit_module = hub.KerasLayer(bit_model_url)

In [ ]:
img, lbl = next(iter(train_ds))
img, lbl = next(iter(val_ds))

In [ ]:
bit_model = keras.Sequential([
                          keras.Input(shape=IMG_SIZE+(3,)),
                          bit_module,
                          keras.layers.Dense(N_CLASSES)
                            ])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=4)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=100,
    decay_rate=.9)

optimizer = keras.optimizers.Adam(lr_schedule)

In [ ]:
bit_model.compile(loss=loss_fn,
                  optimizer=optimizer,
                  metrics=['accuracy'])

In [ ]:
history = bit_model.fit(train_ds,
                        validation_data=val_ds,
                        callbacks=[early_stopping],
                        steps_per_epoch=20,
                        epochs=2,
                        class_weight=class_weight)

Epoch 1/2
20/20 [==============================] - ETA: 0s - loss: 439.1434 - accuracy: 0.1629 